<a href="https://colab.research.google.com/github/tohweizhong/agentic-demos/blob/main/ADK_with_Agent_Engine_Integration_with_AgentSpace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Note
This notebook is not shared with external developers. If you're looking for the notebook shared with external developers, it is here: https://colab.sandbox.google.com/drive/1FKrU9bqJzXQNrnhvXeW6Isp9yxPNmsDC?resourcekey=0-evXn6PctfoF1-jC1CJmgGw

If you have any issues or feature requests, please feel free to comment in https://b.corp.google.com/issues/404890021

Orcas RFC: http://go/orcas-rfc-58

# Use the ADK to define an agent and deploy it to Vertex AI Agent Engine
This notebook shows how to use the Agent Framework to define a simple, hello world, agent for a dice rolling game. Then, we deploy the defined agent to the Vertex AI Agent Engine to call and manage the agent in production. In this notebook you will learn how to do the following:

*  Install Agent Framework
*  Define functions as tools
*  Create a "Hello World" agent using the Agent Framework
*  Deploy the agent on Vertex AI Agent Engine
*  Call the Agent Engine to manage and interact with the agent


# Authenticate
You need this to download the SDK from GCS

In [1]:
from google.colab import auth
auth.authenticate_user()

RELEASE_VERSION='google_adk-0.0.2.dev20250402+nightly743266262'
WHL_FILE = f"{RELEASE_VERSION}-py3-none-any.whl"

# Download and install the ADK and Agent Engine SDK

Download `.whl` file from the shared GCS bucket:

In [2]:
FOLDER = "gs://agent_framework/history/"

!gsutil cp {FOLDER}{WHL_FILE} .
!pip3 install google_adk-0.0.2.dev*.whl --ignore-requires-python --quiet ## for the google-adk in private preview
!pip3 install "google-cloud-aiplatform[agent_engines] @ git+https://github.com/googleapis/python-aiplatform.git@copybara_738852226" --force-reinstall --quiet ## for the prebuilt template

Copying gs://agent_framework/history/google_adk-0.0.2.dev20250402+nightly743266262-py3-none-any.whl...
/ [1 files][551.3 KiB/551.3 KiB]                                                
Operation completed over 1 objects/551.3 KiB.                                    
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

# Initialization

In [3]:
import vertexai
from vertexai import agent_engines

PROJECT_ID = "weizhong-project01" #@param {type:"string"}
LOCATION = "us-central1" #@param {type:"string"}
STAGING_BUCKET = "gs://for-agent-engine-may25" #@param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

# Defining functions
roll_die - rolls dice to create random number

check_prime - checks if a number is prime

In [4]:
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.

  Returns:
    An integer of the result of rolling the die.
  """
  import random

  return random.randint(1, sides)

In [7]:
def check_prime(nums: list[int]) -> list[str]:
  """Check if a given list of numbers are prime.

  Args:
    nums: The list of numbers to check.

  Returns:
    A str indicating which number is prime.
  """

  primes = set()
  for number in nums:
    number = int(number)
    if number <= 1:
      continue
    is_prime = True
    for i in range(2, int(number**0.5) + 1):
      if number % i == 0:
        is_prime = False
        break
    if is_prime:
      primes.add(number)
  return (
      'No prime numbers found.'
      if not primes
      else f"{', '.join(str(num) for num in primes)} are prime numbers."
  )

# Define an agent in the agent framework

An AI agent reasons, plans, and takes actions. The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Create a simple agent. An agent requires two things: a display name and a set of instructions. The instructions should be detailed, making it clear exactly how the agent should behave.



In [8]:
from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp

agent = Agent(
    model="gemini-2.0-flash",
    name='data_processing_agent',
    instruction="""
      You roll dice and answer questions about the outcome of the dice rolls.
      You can roll dice of different sizes.
      You can use multiple tools in parallel by calling functions in parallel(in one request and in one round).
      The only things you do are roll dice for the user and discuss the outcomes.
      It is ok to discuss previous dice roles, and comment on the dice rolls.
      When you are asked to roll a die, you must call the roll_die tool with the number of sides. Be sure to pass in an integer. Do not pass in a string.
      You should never roll a die on your own.
      When checking prime numbers, call the check_prime tool with a list of integers. Be sure to pass in a list of integers. You should never pass in a string.
      You should not check prime numbers before calling the tool.
      When you are asked to roll a die and check prime numbers, you should always make the following two function calls:
      1. You should first call the roll_die tool to get a roll. Wait for the function response before calling the check_prime tool.
      2. After you get the function response from roll_die tool, you should call the check_prime tool with the roll_die result.
        2.1 If user asks you to check primes based on previous rolls, make sure you include the previous rolls in the list.
      3. When you respond, you must include the roll_die result from step 1.
      You should always perform the previous 3 steps when asking for a roll and checking prime numbers.
      You should not rely on the previous history on prime results.
    """,
    tools=[roll_die, check_prime],
)

app = AdkApp(
    agent=agent,
    enable_tracing=True,
    )

app

## Let's try it (locally)

Under the hood, `app: ADKApp` will be using an in-memory session store when running locally.

**Remark**: the operations below (i.e., `.create_session`, `.list_sessions`, `.get_session`, `.stream_query`) are only supported by the `agent_engines.ADKApp` class, not the `google.adk.agents.Agent` class.

In [9]:
session = app.create_session(user_id="wz")
session

Session(id='f121efde-cc01-49fa-a606-cbec4402e6ec', app_name='default-app-name', user_id='wz', state={}, events=[], last_update_time=1750660252.3756783)

In [11]:
app.list_sessions(user_id="wz")

ListSessionsResponse(session_ids=['f121efde-cc01-49fa-a606-cbec4402e6ec'])

In [12]:
session = app.get_session(user_id="wz", session_id=session.id)
session

Session(id='f121efde-cc01-49fa-a606-cbec4402e6ec', app_name='default-app-name', user_id='wz', state={}, events=[], last_update_time=1750660252.3756783)

In [13]:
for event in app.stream_query(
    user_id="wz",
    session_id=session.id,
    message="roll a 20 sided die",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'af-7b6827f0-0a76-4ebd-8901-3250850186de', 'args': {'sides': 20}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-39941ef4-6811-439b-8470-942171bd6b3d', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'XxT9Hg53', 'timestamp': 1750660263.26358}
{'content': {'parts': [{'function_response': {'id': 'af-7b6827f0-0a76-4ebd-8901-3250850186de', 'name': 'roll_die', 'response': {'result': 15}}}], 'role': 'user'}, 'invocation_id': 'e-39941ef4-6811-439b-8470-942171bd6b3d', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'i13v67P8', 'timestamp': 1750660266.007397}
{'content': {'parts': [{'text': 'I rolled a 20-sided die and got a 15.\n'}], 'role': 'model'}, 'invocation_id': 'e-39941ef4-6811-439b-8470-942171bd6b3d', 'author': 'data_processing_agent', 'actio

In [14]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'

for response in app.streaming_agent_run_with_events(agent_context):
    print(response)

Event from an unknown agent: AgentSpace_root_agent, event id: 14076651604820872102
Event from an unknown agent: AgentSpace_root_agent, event id: fyIpxGqF


{'events': [{'content': {'parts': [{'text': 'I am a large language model, trained by Google.'}], 'role': 'model'}, 'invocation_id': 'e-cde64ae0-1be9-4998-a960-c021ba37ae9c', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'huaW6sTG', 'timestamp': 1750660272.965518}]}


In [15]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'

for response in app.streaming_agent_run_with_events(agent_context):
    for event in response.get("events", []):
        print(event)

Event from an unknown agent: AgentSpace_root_agent, event id: 14076651604820872102
Event from an unknown agent: AgentSpace_root_agent, event id: 6AO9uvWn


{'content': {'parts': [{'text': 'I am a large language model, trained by Google.\n'}], 'role': 'model'}, 'invocation_id': 'e-43dec373-8be1-4790-8946-671b80cafc7d', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'rfnzh3r2', 'timestamp': 1750660282.572782}


You can navigate to https://pantheon.corp.google.com/traces/explorer;project=reasoning-engine-test-1 to view the traces for it (they might take some time to show up).
* If you don't have the permissions to view them in Cloud Trace, please follow the instructions in https://cloud.google.com/trace/docs/finding-traces#before_you_begin to grant yourself permission to use Trace Explorer with the project.

# Deploy to Vertex AI Agent Engine
To call and manage ADK applications in production, deploy the `app` to Vertex AI Agent Engine.

In [16]:
remote_app = agent_engines.create(
    app,
    display_name='ae2_Jun25',
    requirements=[
        WHL_FILE,
        "google-cloud-aiplatform[agent_engines] @ git+https://github.com/googleapis/python-aiplatform.git@copybara_738852226",
        "google-genai==1.20.0"
    ],
    extra_packages=[
        WHL_FILE,
    ],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.87.0', 'cloudpickle': '3.1.1'}
    google_adk-0.0.2.dev20250402+nightly743266262-py3-none-any.whl
                                ^
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google_adk-0.0.2.dev20250402+nightly743266262-py3-none-any.whl', 'google-cloud-aiplatform[agent_engines] @ git+https://github.com/googleapis/python-aiplatform.git@copybara_738852226', 'google-genai==1.20.0', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket for-agent-engine-may25
INFO:vertexai.agent_engines:Wrote to gs://for-agent-engine-may25/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://for-agent-engine-may25/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://for-agent-eng

**Remark**: You can ignore the following warnings:
* (i) `WARNING:vertexai.agent_engines:Failed to parse constraint: ` and
* (ii) `WARNING:vertexai.agent_engines:The following requirements are missing: {'cloudpickle'}`

We have auto-heal mechanisms to still reflect them in the final list of requirements: `INFO:vertexai.agent_engines:The final list of requirements`

**Remark**: The automatic provisioning of permissions for Vertex AI Sessions have not rolled out yet, so you'll have to follow https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#service-agent in the meantime to grant "Vertex AI User" (`roles/aiplatform.user`) to the service account in your GCP project

## Let's try it (remotely)

In [17]:
import pprint

pprint.pprint(remote_app.operation_schemas())

[{'api_mode': '',
  'description': 'Get a session for the given user.\n'
                 '\n'
                 '        Args:\n'
                 '            user_id (str):\n'
                 '                Required. The ID of the user.\n'
                 '            session_id (str):\n'
                 '                Required. The ID of the session.\n'
                 '            **kwargs (dict[str, Any]):\n'
                 '                Optional. Additional keyword arguments to '
                 'pass to the\n'
                 '                session service.\n'
                 '\n'
                 '        Returns:\n'
                 '            Session: The session instance (if any). It '
                 'returns None if the\n'
                 '            session is not found.\n'
                 '\n'
                 '        Raises:\n'
                 '            RuntimeError: If the session is not found.\n'
                 '        ',
  'name': 'get

In [18]:
session = remote_app.create_session(user_id="wz")
session

{'events': [],
 'user_id': 'wz',
 'state': {},
 'id': '3468190077249650688',
 'app_name': '7077239688645312512',
 'last_update_time': 1750660639.994698}

In [19]:
remote_app.list_sessions(user_id="wz")

{'session_ids': ['3468190077249650688']}

In [20]:
session = remote_app.get_session(user_id="wz", session_id=session["id"])
session

{'events': [],
 'last_update_time': 1750660639.994698,
 'state': {},
 'id': '3468190077249650688',
 'app_name': '7077239688645312512',
 'user_id': 'wz'}

In [21]:
for event in remote_app.stream_query(
    user_id="wz",
    session_id=session["id"],
    message="roll a 20 sided die",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'af-55bb5c3b-d3b7-4eeb-b0ff-ccf470c61af8', 'args': {'sides': 20}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-c354f5c4-a095-4080-9411-55c6d33fc5a1', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': '9maxs0OK', 'timestamp': 1750660664.174889}
{'content': {'parts': [{'function_response': {'id': 'af-55bb5c3b-d3b7-4eeb-b0ff-ccf470c61af8', 'name': 'roll_die', 'response': {'result': 16}}}], 'role': 'user'}, 'invocation_id': 'e-c354f5c4-a095-4080-9411-55c6d33fc5a1', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'SVRlm20a', 'timestamp': 1750660664.848511}
{'content': {'parts': [{'text': 'I rolled a 20-sided die and got a 16.\n'}], 'role': 'model'}, 'invocation_id': 'e-c354f5c4-a095-4080-9411-55c6d33fc5a1', 'author': 'data_processing_agent', 'actions

In [22]:
for event in remote_app.stream_query(
    user_id="wz",
    session_id=session["id"],
    message="One more time.",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'af-8977a829-f35d-4160-b8ff-c3d47cf9c5ac', 'args': {'sides': 20}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-127ecd13-467a-4835-b41c-bd7e37efbde8', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': '6LXCh4GA', 'timestamp': 1750660698.111266}
{'content': {'parts': [{'function_response': {'id': 'af-8977a829-f35d-4160-b8ff-c3d47cf9c5ac', 'name': 'roll_die', 'response': {'result': 11}}}], 'role': 'user'}, 'invocation_id': 'e-127ecd13-467a-4835-b41c-bd7e37efbde8', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'mrfAqNaF', 'timestamp': 1750660698.636076}
{'content': {'parts': [{'text': 'I rolled a 20-sided die and got an 11.\n'}], 'role': 'model'}, 'invocation_id': 'e-127ecd13-467a-4835-b41c-bd7e37efbde8', 'author': 'data_processing_agent', 'action

In [23]:
for event in remote_app.stream_query(
    user_id="wz",
    session_id=session["id"],
    message="And again.",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'af-abf7ecfa-7b24-42c9-85a2-49631661785a', 'args': {'sides': 20}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-55de41ee-3e3d-4318-90b1-efa56f8e60d9', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'tx4OyUwE', 'timestamp': 1750660730.315948}
{'content': {'parts': [{'function_response': {'id': 'af-abf7ecfa-7b24-42c9-85a2-49631661785a', 'name': 'roll_die', 'response': {'result': 11}}}], 'role': 'user'}, 'invocation_id': 'e-55de41ee-3e3d-4318-90b1-efa56f8e60d9', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'VfeI23Cx', 'timestamp': 1750660730.86297}
{'content': {'parts': [{'text': "I rolled a 20-sided die and got an 11. That's the second time we've rolled an 11 in a row!\n"}], 'role': 'model'}, 'invocation_id': 'e-55de41ee-3e3d-4318-90b1-efa56f

In [24]:
for event in remote_app.stream_query(
    user_id="wz",
    session_id=session["id"],
    message="What is the sum of all the previous rolls?",
):
    print(event)

{'content': {'parts': [{'text': 'The previous rolls were 16, 11, and 11. The sum of these numbers is 38.\n'}], 'role': 'model'}, 'invocation_id': 'e-36d78553-3947-4d8f-a04d-7a9da5496008', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'nJDYfsly', 'timestamp': 1750660746.872054}


In [25]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'

for event in remote_app.streaming_agent_run_with_events(request_json=agent_context):
    print(event)

{'events': [{'content': {'parts': [{'text': 'I am a large language model, trained by Google.'}], 'role': 'model'}, 'invocation_id': 'e-1d461560-9ea9-4eba-99d2-390fe7f84da1', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'xIYK69UN', 'timestamp': 1750660755.145939}]}


# AgentSpace integration

First, submit request to @krzysiekw with your GCP project number and Agentspace app URL
https://docs.google.com/document/d/1aNIgYM_CphUY4ay8oTn3yxUtRSTcn-TSNYkH2jdlyFs

Next, update config to link Agent to Agentspace:

API reference

* [AssistantService.UpdateAssistant](http://google3/google/cloud/discoveryengine/v1main/assistant_service.proto;l=559-560;rcl=742731234)
* [UpdateAssistantRequest](http://google3/google/cloud/discoveryengine/v1main/assistant_service.proto;l=1774-1775;rcl=742731234)
* [Assistant](http://google3/google/cloud/discoveryengine/v1main/assistant.proto;l=24;rcl=742731234)

In [28]:
%%bash
export PROJECT_ID="weizhong-project01" #@param {type:"string"}
export PROJECT_NUMBER="606459840212" #@param {type:"string"}
export REASONING_ENGINE="projects/weizhong-project01/locations/us-central1/reasoningEngines/7077239688645312512" #@param {type:"string"}
export AGENT_DISPLAY_NAME="Dice rolling agent!"
export AGENT_DESCRIPTION="The agent can only answer how it was built."
export AGENT_ID="agent" #@param {type:"string"}
export AS_APP="enterprise-search-17484208_1748420861365" #@param {type:"string"}

curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "x-goog-user-project: ${PROJECT_ID}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant?updateMask=agent_configs -d '{
    "name": "projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant",
    "displayName": "Default Assistant",
    "agentConfigs": [{
      "displayName": "'"${AGENT_DISPLAY_NAME}"'",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "'"${REASONING_ENGINE}"'"
      },
      "toolDescription": "'"${AGENT_DESCRIPTION}"'",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "'"${AGENT_ID}"'"
    }]
  }'

{
  "name": "projects/606459840212/locations/global/collections/default_collection/engines/enterprise-search-17484208_1748420861365/assistants/default_assistant",
  "displayName": "Default Assistant",
  "agentConfigs": [
    {
      "displayName": "Dice rolling agent",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "projects/weizhong-project01/locations/us-central1/reasoningEngines/7077239688645312512"
      },
      "toolDescription": "The agent can only answer how it was built.",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "agent"
    }
  ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1329    0   678  100   651   3867   3713 --:--:-- --:--:-- --:--:--  7594


Finally, try the Agentspace UI:

1. Open up the Agentspace app in GCP console
2. From left menu, click `Integration`
3. Open the URL provided
4. From left menu, select the agent deployed

# Clean up

In [ ]:
remote_app.delete(force=True)

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/349765473021/locations/us-central1/operations/952030937686736896
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/349765473021/locations/us-central1/reasoningEngines/1684495794217943040
